Nội dung bài viết nhằm mục đích giới thiệu về thuật toán Decision Tree trong việc dự đoán kho hàng. Dữ liệu ở đây là file product_train.csv. Nguồn: https://www.kaggle.com/datasets/aditya954/back-order-data-set

# Đọc dữ liệu từ file

In [1]:
import csv
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
#đọc dữ liệu từ file
df = pd.read_csv("E:\Jupyter NoteBook\XGBoost\product_train.csv")
dataset=df.values

Giải thích về Ý nghĩa từng cột trong bảng dữ liệu product_train.csv

- sku: Mã sản phẩm (Product ID) - Định danh duy nhất cho từng sản phẩm trong tập dữ liệu.

- national_inv: Số lượng hàng tồn kho (inventory count) ở mức quốc gia - Đây là số lượng sản phẩm hiện có trong kho hàng.

- lead_time: Thời gian chờ (lead time) để có sẵn hàng hóa - Đây là thời gian mà nhà cung cấp mất để cung cấp hàng sau khi đặt hàng.

- in_transit_qty: Số lượng hàng đang vận chuyển (In transit quantity) - Đây là số lượng sản phẩm đang được vận chuyển từ nhà cung cấp đến kho.

- forecast_3_month: Dự báo doanh số trong 3 tháng tới (3 month forecast for sale) - Đây là dự báo số lượng sản phẩm được dự đoán sẽ được bán trong 3 tháng tới.

- forecast_6_month: Dự báo doanh số trong 6 tháng tới (6 month forecast for sale) - Đây là dự báo số lượng sản phẩm được dự đoán sẽ được bán trong 6 tháng tới.

- forecast_9_month: Dự báo doanh số trong 9 tháng tới (9 month forecast for sale) - Đây là dự báo số lượng sản phẩm được dự đoán sẽ được bán trong 9 tháng tới.

- sales_1_month: Số lượng sản phẩm đã bán trong 1 tháng gần nhất (actual sales figure for last 1 month) - Đây là số lượng sản phẩm thực tế đã được bán trong tháng gần nhất.

- sales_3_month: Số lượng sản phẩm đã bán trong 3 tháng gần nhất (actual sales figure for last 3 months) - Đây là số lượng sản phẩm thực tế đã được bán trong 3 tháng gần nhất.

- sales_6_month: Số lượng sản phẩm đã bán trong 6 tháng gần nhất (actual sales figure for last 6 months) - Đây là số lượng sản phẩm thực tế đã được bán trong 6 tháng gần nhất.

- sales_9_month: Số lượng sản phẩm đã bán trong 9 tháng gần nhất (actual sales figure for last 9 months) - Đây là số lượng sản phẩm thực tế đã được bán trong 9 tháng gần nhất.

- min_bank: Số lượng sản phẩm tối thiểu cần tồn kho (Minimum recommend amount to stock) - Đây là số lượng tối thiểu của sản phẩm mà kho hàng nên tồn trữ để đáp ứng nhu cầu.

- potential_issue: Vấn đề tiềm tàng (Source issue for part identified) - Cột này chỉ ra sự tồn tại của vấn đề tiềm tàng trong nguồn cung cấp cho một phần cụ thể. Giá trị "Yes" cho biết có vấn đề tiềm tàng, trong khi giá trị "No" cho biết không có vấn đề tiềm tàng.

- pieces_past_due: Số lượng mặt hàng quá hạn (Parts overdue from source) - Đây là số lượng sản phẩm đã đặt hàng nhưng chưa được giao đúng hẹn từ nguồn cung cấp.

- perf_6_month_avg: Hiệu suất nguồn cung cấp trong 6 tháng trước đó (Source performance for prior 6-month period) - Đây là chỉ số hiệu suất của nguồn cung cấp dựa trên dữ liệu trong 6 tháng trước.

- perf_12_month_avg: Hiệu suất nguồn cung cấp trong 12 tháng trước đó (Source performance for prior 12-month period) - Đây là chỉ số hiệu suất của nguồn cung cấp dựa trên dữ liệu trong 12 tháng trước.

- local_bo_qty: Số lượng đơn hàng cục bộ chậm trễ (Amount of stock orders overdue) - Đây là số lượng đơn hàng chậm trễ đối với sản phẩm trong kho hàng.

- deck_risk: Rủi ro về bộ phận (Part risk flag) - Cột này chỉ ra rủi ro liên quan đến bộ phận của sản phẩm. Giá trị "Yes" cho biết có rủi ro, trong khi giá trị "No" cho biết không có rủi ro.

- oe_constraint: Rủi ro về ràng buộc đơn đặt hàng (Part risk flag) - Cột này chỉ ra rủi ro liên quan đến ràng buộc của đơn đặt hàng. Giá trị "Yes" cho biết có rủi ro, trong khi giá trị "No" cho biết không có rủi ro.

- ppap_risk: Rủi ro liên quan đến PPAP (Part risk flag) - Cột này chỉ ra rủi ro liên quan đến quy trình phê duyệt sản phẩm trước sản xuất (PPAP). Giá trị "Yes" cho biết có rủi ro, trong khi giá trị "No" cho biết không có rủi ro.

- stop_auto_buy: Dừng mua tự động (Part risk flag) - Cột này chỉ ra rủi ro liên quan đến việc dừng mua hàng tự động. Giá trị "Yes" cho biết có rủi ro, trong khi giá trị "No" cho biết không có rủi ro.

- rev_stop: Dừng xem xét (Part risk flag) - Cột này chỉ ra rủi ro liên quan đến việc dừng xem xét sản phẩm. Giá trị "Yes" cho biết có rủi ro, trong khi giá trị "No" cho biết không có rủi ro.

- went_on_backorder: Sản phẩm thực sự đã hết hàng (Product actually went on backorder) - Đây là giá trị mục tiêu mà mô hình dự đoán, có giá trị 0 hoặc 1. Giá trị 1 cho biết sản phẩm đã hết hàng (đã vào tình trạng backorder), trong khi giá trị 0 cho biết sản phẩm không hết hàng.

Mục tiêu ở đây là xây dựng mô hình dự đoán went_on_backorder dựa trên những cột thông tin còn lại

In [3]:
df

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,3921548,8,12,0,0,0,0,1,1,2,...,0,0.63,0.75,0,No,No,No,Yes,No,No
1,3191009,83,2,33,157,377,603,44,98,148,...,0,0.68,0.66,0,No,No,No,Yes,No,No
2,2935810,8,4,0,0,0,0,0,0,1,...,0,0.73,0.78,0,No,No,No,Yes,No,No
3,2205847,31,4,63,70,160,223,27,90,164,...,0,0.73,0.78,0,No,No,Yes,Yes,No,No
4,4953497,3,12,0,0,0,0,0,0,0,...,0,0.81,0.74,0,No,No,No,Yes,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250073,3129822,74,8,0,0,0,0,6,18,28,...,0,0.87,0.71,0,No,No,No,Yes,No,No
250074,3100168,7,12,0,0,1,4,1,4,5,...,0,0.73,0.79,0,No,No,No,Yes,No,No
250075,3860965,2,8,0,0,0,0,0,0,0,...,0,0.80,0.79,0,No,No,Yes,Yes,No,No
250076,3477768,37,8,1,14,28,49,9,25,49,...,0,0.97,0.95,0,No,No,No,Yes,No,No


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250078 entries, 0 to 250077
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   sku                250078 non-null  int64  
 1   national_inv       250078 non-null  int64  
 2   lead_time          250078 non-null  int64  
 3   in_transit_qty     250078 non-null  int64  
 4   forecast_3_month   250078 non-null  int64  
 5   forecast_6_month   250078 non-null  int64  
 6   forecast_9_month   250078 non-null  int64  
 7   sales_1_month      250078 non-null  int64  
 8   sales_3_month      250078 non-null  int64  
 9   sales_6_month      250078 non-null  int64  
 10  sales_9_month      250078 non-null  int64  
 11  min_bank           250078 non-null  int64  
 12  potential_issue    250078 non-null  object 
 13  pieces_past_due    250078 non-null  int64  
 14  perf_6_month_avg   250078 non-null  float64
 15  perf_12_month_avg  250078 non-null  float64
 16  lo

In [5]:
df['went_on_backorder'].value_counts()

went_on_backorder
No     248463
Yes      1615
Name: count, dtype: int64

In [6]:
df.isnull().sum()

sku                  0
national_inv         0
lead_time            0
in_transit_qty       0
forecast_3_month     0
forecast_6_month     0
forecast_9_month     0
sales_1_month        0
sales_3_month        0
sales_6_month        0
sales_9_month        0
min_bank             0
potential_issue      0
pieces_past_due      0
perf_6_month_avg     0
perf_12_month_avg    0
local_bo_qty         0
deck_risk            0
oe_constraint        0
ppap_risk            0
stop_auto_buy        0
rev_stop             0
went_on_backorder    0
dtype: int64

Những cột có dữ liệu là kiểu object đổi về dạng số, đổi dữ liệu -99 trong các cột thành giá trị trung vị của cột tương ứng

In [7]:
#recode binary variables as 0 / 1 rather than No / Yes
for col in ['potential_issue',
            'deck_risk',
            'oe_constraint',
            'ppap_risk',
            'stop_auto_buy',
            'rev_stop',
            'went_on_backorder']:    
    df[col]=pd.factorize(df[col])[0]

In [8]:
df['perf_12_month_avg'].unique()
df['perf_6_month_avg']=df['perf_6_month_avg'].replace(-99, np.NaN)
df['perf_12_month_avg']=df['perf_12_month_avg'].replace(-99, np.NaN)
df.fillna(df.median(), inplace=True)

In [9]:
df=df.drop(['sku'],axis=1)

In [10]:
df.head()

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,8,12,0,0,0,0,1,1,2,5,...,0,0.63,0.75,0,0,0,0,0,0,0
1,83,2,33,157,377,603,44,98,148,156,...,0,0.68,0.66,0,0,0,0,0,0,0
2,8,4,0,0,0,0,0,0,1,1,...,0,0.73,0.78,0,0,0,0,0,0,0
3,31,4,63,70,160,223,27,90,164,219,...,0,0.73,0.78,0,0,0,1,0,0,0
4,3,12,0,0,0,0,0,0,0,0,...,0,0.81,0.74,0,0,0,0,0,0,0


# Xây dựng mô hình Decision Tree và đánh giá kết quả

## Chia tập dữ liệu thành 2 phần là train và test với tỉ lệ 9:1

In [11]:
from sklearn.model_selection import train_test_split
Y=df['went_on_backorder']
X=df.drop(['went_on_backorder'],axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 1)

## Khởi tạo và huấn luyện mô hình

In [13]:
from sklearn.tree import DecisionTreeClassifier

# Khởi tạo mô hình Decision Tree
DT = DecisionTreeClassifier()

# Huấn luyện mô hình trên tập huấn luyện
DT.fit(X_train, Y_train)

DecisionTreeClassifier()

## Đánh giá kết quả 

In [14]:
acc = DT.score(X_test, Y_test)
print(acc*100)

99.00831733845169


Hiệu suất của thuật toán đạt 99% cao hơn so với thuật toán RandomForest và XGBoost. Điều này có lẽ bởi vì việc dự báo ở đây khá là đơn giản chỉ trả về kết quả là 0/1